In [1]:
import sys
sys.path.append("..")

In [2]:
import ast
import time
import pandas as pd
import numpy as np

import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import gensim
import torchtext
from torchtext import data
from torchtext import vocab

from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV

from utils.db_client import DBClient

In [3]:
# Make DB Connection
db = DBClient(secrets_path='../configs/db_secrets.json')

I'm trying
Connected to political tweets DB


### Model definition and tuning

In [4]:
# Load data
examples = db.read("select tweet_text_clean, label from staging.train_twitter140 order by Random() limit 100000")

In [7]:
# Define Fields
txt_field = data.Field(sequential=True, 
                       include_lengths=True, 
                       use_vocab=True)
label_field = data.Field(sequential=False, 
                         use_vocab=False, 
                         pad_token=None, 
                         unk_token=None)
train_val_fields = [
    ('SentimentText', txt_field), # process it as text
    ('Sentiment', label_field) # process it as label
]

# Convert text ecamples to Example datatype
examples = [data.Example.fromlist(((ast.literal_eval(example[0])), example[1]), train_val_fields) for example in examples]

# Create dataset
dataset = data.Dataset(examples, train_val_fields)

In [8]:
MAX_VOCAB_SIZE = 25000

# Make vocab for text and label fields
txt_field.build_vocab(dataset,
                      max_size = MAX_VOCAB_SIZE, 
                      vectors = vocab.Vectors('glove.twitter.27B.50d.txt', '../model/glove.twitter.27B/'))
label_field.build_vocab(dataset)

pretrained_embeddings = txt_field.vocab.vectors

# Make train/val/test splits
train_data, test_data, valid_data = dataset.split([0.7, 0.2, 0.1])

In [9]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 7000
Number of validation examples: 2000
Number of testing examples: 1000


In [10]:
# Make data iterators
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), 
                                                          batch_sizes = (BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
                                                                        sort_key = lambda x: len(x.SentimentText),
                                                                        sort_within_batch=True,
                                                                        repeat = False)

In [29]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, pad_idx):#, optimizer, criterion):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.embedding_dim = embedding_dim
        self.pad_idx = pad_idx
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           #bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.optimizer = None #optimizer #optim.Adam(model.parameters())
        self.criterion = None #criterion #nn.BCEWithLogitsLoss()
        
    
    def set_pretrained_weights(self, pretrained_embeddings, txt_field):
        self.embedding.weight.data.copy_(pretrained_embeddings)

        UNK_IDX = txt_field.vocab.stoi[txt_field.unk_token]

        self.embedding.weight.data[UNK_IDX] = torch.zeros(self.embedding_dim)
        self.embedding.weight.data[self.pad_idx] = torch.zeros(self.embedding_dim)
    
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden.squeeze(0))
    
    def binary_accuracy(self, preds, y):
        """
        Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
        """

        #round predictions to the closest integer
        rounded_preds = torch.round(torch.sigmoid(preds))
        correct = (rounded_preds == y).float() #convert into float for division 
        acc = correct.sum() / len(correct)
        return acc
    
    def train_epoch(self, iterator):
        epoch_loss = 0
        epoch_acc = 0

        self.train()
        optimizer = self.optimizer
        for batch in iterator:
            if len(batch) == BATCH_SIZE:
            
                optimizer.zero_grad()

                text, text_lengths = batch.SentimentText
                predictions = self.forward(text, text_lengths).squeeze(1)
                loss = self.criterion(predictions, batch.Sentiment.float())

                acc = self.binary_accuracy(predictions, batch.Sentiment.float())

                loss.backward()

                optimizer.step()

                epoch_loss += loss.item()
                epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)

    def evaluate(self, iterator):

        epoch_loss = 0
        epoch_acc = 0

        model.eval()

        with torch.no_grad():

            for batch in iterator:
                if len(batch) == BATCH_SIZE:

                    text, text_lengths = batch.SentimentText

                    predictions = self.forward(text, text_lengths).squeeze(1)

                    loss = self.criterion(predictions, batch.Sentiment.float())

                    acc = self.binary_accuracy(predictions, batch.Sentiment.float())

                    epoch_loss += loss.item()
                    epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
    def epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

In [26]:
def epoch_training(N_EPOCHS, model, train_iterator, valid_iterator):
    best_valid_loss = float('inf')

    for epoch in range(N_EPOCHS):

        start_time = time.time()

        train_loss, train_acc = model.train_epoch(train_iterator)
        valid_loss, valid_acc = model.evaluate(valid_iterator)

        end_time = time.time()

        epoch_mins, epoch_secs = model.epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'tut2-model.pt')

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        
    return model

In [33]:
optimizers = ["Adam", "Adagrad"]
criterions = ["BCEWithLogitsLoss"] 
learning_rates = [0.001, 0.01]
epochs = [5, 50, 100]
weight_decay = [0, 0.01]

In [39]:
INPUT_DIM = len(txt_field.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
DROPOUT = 0.5
PAD_IDX = txt_field.vocab.stoi[txt_field.pad_token]


best_model_tuple = None
best_accuracy = -1
for optimizer in optimizers:
    # Make a new model at the start
    model = LSTM(INPUT_DIM, 
                EMBEDDING_DIM, 
                HIDDEN_DIM, 
                OUTPUT_DIM, 
                N_LAYERS,
                DROPOUT, 
                PAD_IDX)
    model.set_pretrained_weights(pretrained_embeddings, txt_field)
    for learning_rate in learning_rates:
        for decay in weight_decay:
            # Set OPTIMIZER
            if optimizer == "Adam":
                model.optimizer = optim.Adam(params = model.parameters(), lr = learning_rate, weight_decay = decay) 
            else: 
                model.optimizer = optim.Adagrad(params = model.parameters(), lr = learning_rate, weight_decay = decay)

            for criterion in criterions:
                # SET CRITERION
                if criterion == "BCEWithLogitsLoss":
                    model.criterion = nn.BCEWithLogitsLoss() 
                else: 
                    model.criterion = nn.NLLLoss()
                
                for epoch in epochs:
                    model_name = "-".join([optimizer, str(learning_rate), str(decay), str(criterion), str(epoch)])

                    print(f"working on model {model_name}")
                    trained_model = epoch_training(epoch, model, train_iterator, valid_iterator)
                    _, acc = trained_model.evaluate(valid_iterator)
                    
                    if acc > best_accuracy:
                        best_accuracy = acc
                        best_model_tuple = (model_name, trained_model)


working on model Adam-0.001-0-CrossEntropy-1


RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [31]:
best_model_name = best_model_tuple[0]
best_model = best_model_tuple[1]

In [32]:
print(best_model_name)
best_model.evaluate(test_iterator)

Adam-0.01-0-BCEWithLogitsLoss-1


(0.5763356722891331, 0.62890625)

### Subject Analysis

In [ ]:
# def binary_accuracy(preds, y):
#     """
#     Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
#     """

#     #round predictions to the closest integer
#     rounded_preds = torch.round(torch.sigmoid(preds))
#     correct = (rounded_preds == y).float() #convert into float for division 
#     acc = correct.sum() / len(correct)
#     return acc

In [ ]:
# def train(model, iterator):
    
#     epoch_loss = 0
#     epoch_acc = 0
    
#     model.train()
    
#     for batch in iterator:
        
#         optimizer.zero_grad()
        
#         text, text_lengths = batch.SentimentText
        
#         predictions = model(text, text_lengths).squeeze(1)

#         loss = model.criterion(predictions, batch.Sentiment.float())
        
#         acc = binary_accuracy(predictions, batch.Sentiment.float())
        
#         loss.backward()
        
#         model.optimizer.step()
        
#         epoch_loss += loss.item()
#         epoch_acc += acc.item()
        
#     return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# def evaluate(model, iterator):
    
#     epoch_loss = 0
#     epoch_acc = 0
    
#     model.eval()
    
#     with torch.no_grad():
    
#         for batch in iterator:
            
#             text, text_lengths = batch.SentimentText

#             predictions = model(text, text_lengths).squeeze(1)
            
#             loss = model.criterion(predictions, batch.Sentiment.float())
            
#             acc = binary_accuracy(predictions, batch.Sentiment.float())

#             epoch_loss += loss.item()
#             epoch_acc += acc.item()
        
#     return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
# def epoch_time(start_time, end_time):
#     elapsed_time = end_time - start_time
#     elapsed_mins = int(elapsed_time / 60)
#     elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
#     return elapsed_mins, elapsed_secs

In [ ]:
# INPUT_DIM = len(txt_field.vocab)
# EMBEDDING_DIM = 50
# HIDDEN_DIM = 256
# OUTPUT_DIM = 1
# N_LAYERS = 2
# DROPOUT = 0.5
# PAD_IDX = txt_field.vocab.stoi[txt_field.pad_token]
# OPTIMIZER = optim.Adam(model.parameters())
# CRITERION = nn.BCEWithLogitsLoss()

# model = LSTM(INPUT_DIM, 
#             EMBEDDING_DIM, 
#             HIDDEN_DIM, 
#             OUTPUT_DIM, 
#             N_LAYERS,
#             DROPOUT, 
#             PAD_IDX,
#             OPTIMIZER,
#             CRITERION)

In [ ]:
trained_model = epoch_training(10, model, train_iterator, valid_iterator)

In [ ]:
#OPTIMIZER = optim.Adam(model.parameters())
#CRITERION = nn.BCEWithLogitsLoss()

# model = LSTM(INPUT_DIM, 
#             EMBEDDING_DIM, 
#             HIDDEN_DIM, 
#             OUTPUT_DIM, 
#             N_LAYERS,
#             DROPOUT, 
#             PAD_IDX,
#             OPTIMIZER,
#             CRITERION)

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

NameError: name 'model' is not defined